In [1]:
#건축,공예,회화,서예,장신구,복식,과학
from retry import retry
import anthropic
import json
from pathlib import Path


In [4]:

client = anthropic.Anthropic()

def load_relics_data(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"파일 읽기 오류: {e}")
        return []


@retry(tries=5, delay=1)
def request_model(prompt):

    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        temperature=0,
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<json>"},
        ],
        stop_sequences=["</json>"]
    )
    # print(message.content[0].text)
    return json.loads(message.content[0].text)
    #print(message)


# 파일 경로 설정
file_path = Path("combined_relics.json")

# 데이터 로드
relics_data = load_relics_data(file_path)

relic_prompt ="""
{relic_info}
```
위의 유물 정보를 읽고 다음으로 답할 것:
- 국적, 시대는,지정문화유산은 원문의 네용을 그대로 할당할 것. 작품 종류의 경우 하나 이상 True/False를 기입할 것
- <json> 태그로 감싸 다음 JSON 포맷으로 응답할 것
{{"국적":, "시대":, "지정문화유산":, "건축":, "조각":, "공예":, "회화":, "서예":, "장신구":, "복식":, "과학기술":, "불교":}}
"""

# for relic_id, relic_info in relics_data.items():
#     _relic_info_ = f"{relic_info['label']}, {relic_info['content']}"
#     _relic_prompt_ = relic_prompt.format(relic_info=_relic_info_)
#     #print(_relic_prompt_)
#     message = request_model(_relic_prompt_)
#     print(relic_id)
#     relics_data[relic_id]['search'] = message


# output_path = "relics_index.json"
# with open(output_path, "w", encoding="utf-8") as f:
#     json.dump(relics_data, f, ensure_ascii=False, indent=4)


In [5]:
import json

output_path = "relics_index.json"
with open(output_path, "r", encoding="utf-8") as f:
    relics_index = json.load(f)


relics_index["348"]["search"]

{'국적': '한국',
 '시대': '통일신라',
 '지정문화유산': '국보',
 '건축': False,
 '조각': True,
 '공예': False,
 '회화': False,
 '서예': False,
 '장신구': False,
 '복식': False,
 '과학기술': False,
 '불교': True}

In [7]:

import pandas as pd

# 각 아이템의 search 필드를 추출하여 데이터프레임 생성
search_data = {}
for relic_id, relic_info in relics_index.items():
    if 'search' in relic_info:
        search_data[relic_id] = relic_info['search']

# 데이터프레임으로 변환
search_df = pd.DataFrame.from_dict(search_data, orient='index')
search_df.reset_index(inplace=True)
search_df.rename(columns={'index': 'relic_id'}, inplace=True)

# 결과 확인
search_df.head()

,relic_id,국적,시대,지정문화유산,건축,조각,공예,회화,서예,장신구,복식,과학기술,불교
0,348,한국,통일신라,국보,False,True,False,False,False,False,False,False,True
1,349,한국,통일신라,국보,False,True,False,False,False,False,False,False,True
2,350,한국,고려,보물,False,False,True,False,False,False,False,False,False
3,355,한국,조선,국보,False,False,True,False,False,False,False,False,False
4,358,한국,고려,보물,False,False,True,False,False,False,False,False,False


In [8]:
# 시대가 조선, 불교가 True, 조각이 True인 문화재 찾기
search_df[(search_df["시대"] == "조선") & (search_df["불교"] == True) & (search_df["조각"] == True)]

,relic_id,국적,시대,지정문화유산,건축,조각,공예,회화,서예,장신구,복식,과학기술,불교
136,2350,한국,조선,보물,False,True,False,False,False,False,False,False,True
252,4875,한국,조선,보물,False,True,False,False,False,False,False,False,True
337,36510223,한국,조선,보물,False,True,False,False,False,False,False,False,True


In [14]:
tools = [
    {
        "name": "search_wiki",
        "description": "위키백과 검색",
        "input_schema": {
            "type": "object",
            "properties": {
                "keywards": {"type": "string", "description": "검색 키워드"},
            },
        },
    },
    {
        "name": "search_artifacts",
        "description": "박물관 전시품 검색",
        "input_schema": {
            "type": "object",
            "properties": {
                "period": {  # 시대
                    "type": "string",
                    "enum": ["조선","고려","통일신라","신라","일제강점","삼국","백제","Not Specified"],
                },
                "designation": {  # 지정문화유산
                    "type": "string",
                    "enum": ["국보", "보물", "Not Specified"],
                },
                "architecture": {  # 건축
                    "type": "boolean",
                    "description": "건축 문화재 여부",
                },
                "sculpture": {  # 조각
                    "type": "boolean",
                    "description": "조각 문화재 여부",
                },
                "craft": {"type": "boolean", "description": "공예 문화재 여부"},  # 공예
                "painting": {  # 회화
                    "type": "boolean",
                    "description": "회화 문화재 여부",
                },
                "calligraphy": {  # 서예
                    "type": "boolean",
                    "description": "서예 문화재 여부",
                },
                "accessories": {  # 장신구
                    "type": "boolean",
                    "description": "장신구 문화재 여부",
                },
                "clothing": {  # 복식
                    "type": "boolean",
                    "description": "복식 문화재 여부",
                },
                "science": {  # 과학기술
                    "type": "boolean",
                    "description": "과학기술 관련 문화재 여부",
                },
                "buddhism": {  # 불교
                    "type": "boolean",
                    "description": "불교 관련 문화재 여부",
                },
                "buddhism": {  # 불교
                    "type": "boolean",
                    "description": "불교 관련 문화재 여부",
                },
            },
            "required": [
                "period","designation","architecture","sculpture","craft","painting",
                "calligraphy","accessories","clothing","science","buddhism"
            ],
        },
    },
    {
        "name": "talk_to_docent",
        "description": "위의 도구와 무관한 내용에 대한 응답",
        "input_schema": {
            "type": "object",
            "properties": {
                "user_message": {"type": "string", "description": "사용자의 메시지"},
            },
        },
    },
]

In [18]:

def get_required_tool(query, tool_name):
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        temperature=0,
        tools=tools,
        tool_choice={"type": "any"},
        messages=[{"role": "user","content": query}]
    )
    for block in response.content:
        if block.type == "tool_use" and block.name == tool_name:
            return block
        
    return None, response

# 사용 예시
search_query = "그림에 대해 어떻게 생각해?"
required_tool, response = get_required_tool(search_query, "search_artifacts")
print(required_tool)

None


In [19]:
response

Message(id='msg_011bYPabCsA1X3t1DTrwvq6a', content=[ToolUseBlock(id='toolu_012sQEk8PuNspA5UmGwNriFf', input={'user_message': '그림에 대해 어떻게 생각해?'}, name='talk_to_docent', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=1291, output_tokens=58))

In [121]:
# required_tools = []
# for block in result.content:
#     print(block.type)
#     if block.type == "tool_use":        
#         print("tool_name",block.name)        
#         print("tool_input",block.input)
#         required_tools.append(block) 
        


In [123]:
required_tool.model_dump()


{'id': 'toolu_013wgjAZRosdce87YMAhszKu',
 'input': {'period': '조선',
  'designation': 'Not Specified',
  'architecture': False,
  'sculpture': True,
  'craft': False,
  'painting': False,
  'calligraphy': False,
  'accessories': False,
  'clothing': False,
  'science': False,
  'buddhism': True},
 'name': 'search_artifacts',
 'type': 'tool_use'}

In [124]:
required_tool.input

{'period': '조선',
 'designation': 'Not Specified',
 'architecture': False,
 'sculpture': True,
 'craft': False,
 'painting': False,
 'calligraphy': False,
 'accessories': False,
 'clothing': False,
 'science': False,
 'buddhism': True}

In [129]:
# 영문 결과를 한국어로 변환하는 함수
def convert_to_korean(result):
    return {
        '국적': '한국',  # 기본값으로 한국 설정
        '시대': result['period'],
        '지정문화유산': result['designation'],
        '건축': result['architecture'],
        '조각': result['sculpture'],
        '공예': result['craft'],
        '회화': result['painting'],
        '서예': result['calligraphy'],
        '장신구': result['accessories'],
        '복식': result['clothing'],
        '과학기술': result['science'],
        '불교': result['buddhism']
    }

# 한국어로 변환
search_condition = convert_to_korean(required_tool.input)
pprint(search_condition)


{'건축': False,
 '공예': False,
 '과학기술': False,
 '국적': '한국',
 '복식': False,
 '불교': True,
 '서예': False,
 '시대': '조선',
 '장신구': False,
 '조각': True,
 '지정문화유산': 'Not Specified',
 '회화': False}


In [131]:
def search_artifacts(relics_index, search_condition):
    # 검색 조건에서 'Not Specified'가 아닌 항목만 필터링
    valid_conditions = {
        k: v for k, v in search_condition.items() 
        if v != 'Not Specified'
    }    
    results = {}    
    condition_keys = valid_conditions.keys()    
    for artifact_id, artifact_data in relics_index.items():
       
        search_data = artifact_data['search']
        
        if all(
            search_data.get(key) == valid_conditions[key]
            for key in condition_keys
        ):
            results[artifact_id] = artifact_data
            
    return results


matched_artifacts = search_artifacts(relics_index, search_condition)
print(f"검색된 항목 수: {len(matched_artifacts)}")

# 결과 출력
for artifact_id, artifact_data in matched_artifacts.items():
    print(f"ID: {artifact_id}")
    print(f"명칭: {artifact_data['label'].get('명칭', 'N/A')}")
    print("-" * 50)


검색된 항목 수: 3
ID: 2350
명칭: 「정덕십년」이 새겨진 석조 지장보살좌상
--------------------------------------------------
ID: 4875
명칭: 봉인사 부도암 사리탑(석관탑비)
--------------------------------------------------
ID: 36510223
명칭: 지장암 목조비로자나불좌상
--------------------------------------------------
